Working with REST APIs 
CoinGeckoAPI

In [1]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
bitcoin_Data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='inr' , days = 30)

In [7]:
bitcoin_Data['prices'][:3]

[[1667466902459, 1681593.8618606324],
 [1667470351966, 1683664.9081504168],
 [1667473998279, 1678814.8985645408]]

Converting the lists to a DataFrame

In [ ]:
import pandas as pd
data = pd.DataFrame(bit)